In [ ]:
from datetime import datetime

import pandas as pd
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

from src.scrape.scrape_users import scrape_user_tooltip_selenium
from src.scrape.get_source_page import get_source_selenium
from src.load.load_data_to_postgresdb import load_user_to_staging, load_user_staging_to_production
from src.utilities.database_utils import connect

Define varirables

In [ ]:
host = 'https://voz.vn'
page = 1
users_data = []

Start scraping users

In [ ]:
while True:
    url = f"{host}/online/?type=member&page={page}"
    driver = get_source_selenium(url)
    action_chain = ActionChains(driver)
    users = driver.find_elements(By.CLASS_NAME, "username")
    for user in users:
        try:
            action_chain.move_to_element(user).pause(1).perform()
        except:
            pass
        # print(user.text)
    tooltips = driver.find_elements(By.CSS_SELECTOR, 'div[role="tooltip"]')
    users_data.extend([scrape_user_tooltip_selenium(tooltip) for tooltip in tooltips])
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "Next"))
        )
        page += 1
    except:
        print("Next button not found")
        break
    finally:
        driver.quit()

Remove `Null` value and add field `updated_at`

In [ ]:
res = [i for i in users_data if i is not None]
df = pd.DataFrame(res)
df.drop_duplicates(subset="user_id", keep="last", inplace=True)
df['updated_at'] = datetime.now().strftime("%b %d, %Y %I:%M %p")

Loading data into database

In [ ]:
engine = connect(section="postgresql_aws")

In [ ]:
load_user_to_staging(engine, "users_staging", df)

In [ ]:
load_user_staging_to_production(engine, staging_table="users_staging", prod_table="users")

test

In [ ]:
# engine.execute("select count(user_id) from users").fetchone()